In [5]:
import angr

proj = angr.Project('./binaries/Diplomacy is Not an Option.exe',  auto_load_libs=False)

cfg = proj.analyses.CFG()

dict(proj.kb.functions)

{5368713216: <Function UnityMain (0x140001000)>,
 5368713224: <Function sub_140001008 (0x140001008)>,
 5368713408: <Function sub_1400010c0 (0x1400010c0)>,
 5368713424: <Function sub_1400010d0 (0x1400010d0)>,
 5368713748: <Function sub_140001214 (0x140001214)>,
 5368713824: <Function _start (0x140001260)>,
 5368713844: <Function sub_140001274 (0x140001274)>,
 5368713904: <Function sub_1400012b0 (0x1400012b0)>,
 5368713980: <Function sub_1400012fc (0x1400012fc)>,
 5368714120: <Function sub_140001388 (0x140001388)>,
 5368714265: <Function sub_140001419 (0x140001419)>,
 5368714272: <Function sub_140001420 (0x140001420)>,
 5368714308: <Function sub_140001444 (0x140001444)>,
 5368714352: <Function sub_140001470 (0x140001470)>,
 5368714412: <Function sub_1400014ac (0x1400014ac)>,
 5368714436: <Function _security_init_cookie (0x1400014c4)>,
 5368714608: <Function sub_140001570 (0x140001570)>,
 5368714612: <Function sub_140001574 (0x140001574)>,
 5368714620: <Function sub_14000157c (0x14000157c